In [ ]:
import os
import shutil
from google.colab import drive 
drive.mount('/content/gdrive')
folder_path = "../content/bert-finetuned-sem_eval-english"
print(os.listdir(folder_path))
shutil.rmtree('../content/bert-finetuned-sem_eval-english')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['checkpoint-256', 'checkpoint-742', 'checkpoint-265', 'checkpoint-861', 'checkpoint-17', 'checkpoint-480', 'checkpoint-492', 'checkpoint-60', 'checkpoint-369', 'checkpoint-1353', 'checkpoint-100', 'checkpoint-424', 'checkpoint-187', 'checkpoint-384', 'checkpoint-689', 'checkpoint-128', 'checkpoint-53', 'checkpoint-64', 'checkpoint-246', 'checkpoint-34', 'checkpoint-1599', 'checkpoint-119', 'checkpoint-140', 'checkpoint-530', 'checkpoint-40', 'checkpoint-153', 'checkpoint-318', 'checkpoint-1230', 'checkpoint-170', 'checkpoint-238', 'checkpoint-106', 'checkpoint-636', 'checkpoint-70', 'checkpoint-192', 'checkpoint-150', 'runs', 'checkpoint-255', 'checkpoint-442', 'checkpoint-615', 'checkpoint-90', 'checkpoint-477', 'checkpoint-408', 'checkpoint-984', 'checkpoint-795', 'checkpoint-123', 'checkpoint-476', 'checkpoint-510', 'checkpoint-10', 'checkpoint-20', 'ch

In [ ]:
import pandas as pd
import os
!pip install datasets
import torch
from datasets import *
!pip install transformers==4.28.0
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, EvalPrediction, AutoTokenizer
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#helper functions

def preprocess_data(examples):
  # take a batch of texts
  text = examples["new_text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"torch.device: {device}")

torch.device: cuda


In [ ]:
    from google.colab import drive 
    drive.mount('/content/gdrive')
    folder_path = "gdrive/MyDrive/data/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# get list of files in folder
file_list = os.listdir(folder_path)

# create empty list to hold dataframes
df_list = []

# loop through files in folder
for file in file_list:
    # check if file is a CSV
    if file.endswith('.csv') and file.startswith('utterances_clean'):
        # read CSV file into a pandas dataframe
        df = pd.read_csv(os.path.join(folder_path, file))
        # append dataframe to list
        df_list.append(df)

# concatenate all dataframes in list into a single dataframe
df = pd.concat(df_list, axis=0, ignore_index=True)

import numpy as np

df = df.merge(
    df[['speaker', 'side']].drop_duplicates(ignore_index=True),
    how='left',
    left_on='speaker_addressed',
    right_on='speaker',
    suffixes=('', '_addressed')).drop('speaker_addressed', axis=1)

for name_col, type_col in {'speaker': 'speaker_type', 'speaker_replied_to': 'speaker_type_replied_to'}.items():
    df[f'{name_col}_natural'] = df[name_col].apply(lambda name: ' '.join(name.lstrip('j__').split('_')).title())
    speaker_type_translation = {
        'J': 'Justice',
        'A': 'Attorney',
        '<Inaudile>': None
    }
    # df[f'{name_col}_natural'].replace(to_replace='<Inaudible>', value=None, inplace=True)

    df[type_col].fillna('na', inplace=True)
    df[f'{type_col}_natural'] = df[type_col].apply(lambda s_type: speaker_type_translation[s_type] if not s_type == 'na' else None)

    df[f'{name_col}_natural'] = df[f'{type_col}_natural'] + ' ' + df[f'{name_col}_natural']
    df[f'{name_col}_natural'].fillna('Unknown', inplace=True)
    df.drop(f'{type_col}_natural', axis=1, inplace=True)

side_translation = {
    0: 'Responding',
    1: 'Petitioning',
    2: '',
    3: ''
}

df['side'].fillna(3, inplace=True)
df[f'side_natural'] = df['side'].apply(lambda side: side_translation[side])

df['side_addressed'].fillna(3, inplace=True)
df[f'side_addressed_natural'] = df['side_addressed'].apply(lambda side: side_translation[side])


df["new_text"] = "<UTTERANCE_START>" + df['side_natural'] + " " + df["speaker_natural"] + " says: '" + df["text"] + "' to " + df['side_addressed_natural'] + " " + df["speaker_replied_to_natural"] + " <UTTERANCE_END>"


j_columns = [col for col in df.columns if col.startswith('votes_side_j_')]



<ipython-input-6-72303f6b461b>:12: DtypeWarning: Columns (17,60,62,64,66,68,70,72,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-6-72303f6b461b>:12: DtypeWarning: Columns (58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-6-72303f6b461b>:12: DtypeWarning: Columns (58,60,62,64,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-6-72303f6b461b>:12: DtypeWarning: Columns (17,56,58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))


In [ ]:
NB_JUSTICIES = 15

nb_cases = {}
for col in j_columns:
    nb_cases[col] = df.drop_duplicates(
        subset='case_id',
        keep='first'
    )[col].count()

frequent_justices = list(
    map(
        lambda pair: pair[0],
        sorted(
            nb_cases.items(),
            key = lambda pair: pair[1], reverse=True
        )
    )
)[10:15]#[:NB_JUSTICIES]

frequent_justices

['votes_side_j__elena_kagan',
 'votes_side_j__sandra_day_oconnor',
 'votes_side_j__william_h_rehnquist',
 'votes_side_j__neil_gorsuch',
 'votes_side_j__brett_m_kavanaugh']

In [ ]:
targets = frequent_justices + ['win_side']

In [ ]:
import random
max_length = 3000

def select_random_chars(string):
    string_length = len(string)

    # Check if the string is long enough to select 3000 characters
    if string_length <= max_length:
        return string

    # Generate a random starting index
    start_index = random.randint(0, string_length - max_length)

    # Select the 3000 characters from the string
    selected_chars = string[start_index:start_index + max_length]
    
    return selected_chars

In [ ]:
EPOCHS = 15

# uncomment next line if want to try code with only one judge, judge 8 works well
#justice = j_columns[8]

results = pd.DataFrame()

for justice in targets:
    df_j = df[['case_id', 'new_text'] + [justice]]
    grouped_df = df_j.groupby('case_id')['new_text'].apply(lambda x: ','.join(x)).reset_index()
    justices = df[["case_id"] + [justice]].drop_duplicates(keep='first')

    df1 = pd.merge(grouped_df, justices, left_on='case_id', right_on='case_id', how='left').dropna(axis='rows', how='any')
    df1 = df1.drop(df1[~df1[justice].isin([0, 1])].index)

    print(df1[justice].value_counts(normalize=True))

    df1['new_text'] = df1['new_text'].apply(select_random_chars)

    # if there's no data for this judge go to next judge in the loop
    if len(df1) < 10:
        str = justice + " judge does not have enough cases for the current dataset"
        print(str)
        continue

    dataset = Dataset.from_pandas(df1.drop('case_id', axis=1), preserve_index = False)

    dataset = dataset.train_test_split(test_size=0.3, shuffle=True)

    dataset_test_valid = dataset['test'].train_test_split(test_size=0.5, shuffle=True)

    dataset = DatasetDict({
        'train': dataset['train'],
        'test': dataset_test_valid['test'],
        'validation': dataset_test_valid['train']})

    labels = [label for label in dataset['train'].features.keys() if label not in ['case_id', 'new_text']]
    id2label = {idx:label for idx, label in enumerate(labels)}
    label2id = {label:idx for idx, label in enumerate(labels)}

    tokenizer = AutoTokenizer.from_pretrained(
        "bert-base-uncased",
        return_overflowing_tokens=True
    )

    encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                                problem_type="multi_label_classification", 
                                                                num_labels=len(labels),
                                                                id2label=id2label,
                                                                label2id=label2id)

    batch_size = 8
    metric_name = "f1"

    args = TrainingArguments(
        f"bert-finetuned-sem_eval-english",
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=EPOCHS,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model=metric_name,
        #push_to_hub=True,
    )

    os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'
    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    trainer.train()

    val_metrics = trainer.evaluate()
    val_metrics = dict((key.replace('eval', 'val'), [value]) for (key, value) in val_metrics.items())

    # Initialize a new trainer instance with the trained model and test data
    trainer_test = Trainer(
        model=model,  
        args=args,
        eval_dataset=encoded_dataset["test"], 
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
        )

    # Evaluate the model on the test data
    test_metrics = trainer_test.evaluate()
    test_metrics = dict((key.replace('eval', 'test'), [value]) for (key, value) in test_metrics.items())

    metrics = val_metrics
    metrics.update(test_metrics)
    metrics['justice'] = [justice]

    results = pd.concat([results, pd.DataFrame(metrics)], ignore_index=True)

results = results[
    ['justice', 'epoch'] +
    [col for col in results.columns if col.startswith('val_')] +
    [col for col in results.columns if col.startswith('test_')]
]

1.0    0.606612
0.0    0.393388
Name: votes_side_j__elena_kagan, dtype: float64


Map:   0%|          | 0/423 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

...

Map:   0%|          | 0/980 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.619115,0.704762,0.500000,0.704762
2,No log,0.627005,0.709524,0.508065,0.709524
3,No log,0.627730,0.695238,0.502616,0.695238
4,No log,0.732293,0.633333,0.486813,0.633333
5,0.635000,1.032288,0.542857,0.474172,0.542857
6,0.635000,1.690379,0.547619,0.496295,0.547619
7,0.635000,1.913510,0.609524,0.498038,0.609524
8,0.635000,2.347598,0.576190,0.535310,0.576190
9,0.171300,2.433172,0.566667,0.486378,0.566667
10,0.171300,2.455276,0.590476,0.512642,0.590476


In [ ]:
results.to_csv("gdrive/MyDrive/data/BERT_results.csv", index=False)
results


,justice,epoch,val_loss,val_f1,val_roc_auc,val_accuracy,val_runtime,val_samples_per_second,val_steps_per_second,test_loss,test_f1,test_roc_auc,test_accuracy,test_runtime,test_samples_per_second,test_steps_per_second
0,votes_side_j__elena_kagan,15.0,0.629866,0.747253,0.500000,0.747253,0.7347,123.860,16.333,0.679129,0.571429,0.500000,0.571429,0.6918,131.546,17.347
1,votes_side_j__sandra_day_oconnor,15.0,0.564268,0.758621,0.500000,0.758621,0.4865,119.222,16.444,0.629993,0.677966,0.500000,0.677966,0.4469,132.019,17.901
2,votes_side_j__william_h_rehnquist,15.0,0.648236,0.636364,0.583333,0.636364,0.4768,115.363,14.683,0.671953,0.545455,0.437500,0.545455,0.4193,131.163,16.693
3,votes_side_j__neil_gorsuch,15.0,0.667354,0.620690,0.500000,0.620690,0.2665,108.818,15.009,0.648137,0.633333,0.500000,0.633333,0.2358,127.203,16.960
4,votes_side_j__brett_m_kavanaugh,15.0,0.657183,0.647059,0.500000,0.647059,0.1820,93.413,16.485,0.690969,0.611111,0.500000,0.611111,0.1504,119.651,19.942
5,win_side,15.0,0.627005,0.709524,0.508065,0.709524,1.6644,126.174,16.222,0.635071,0.680952,0.496528,0.680952,1.5946,131.694,16.932
